In [1]:
print("OK")

OK


In [5]:
%pwd

'c:\\Users\\KIIT0001\\Documents\\Medical_chatbot\\research'

In [1]:
import os
os.chdir("../")

In [7]:
%pwd

'c:\\Users\\KIIT0001\\Documents\\Medical_chatbot'

In [2]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [4]:
def load_pdf_file(data):
    loader=DirectoryLoader(data,
                           glob="*.pdf",
                           loader_cls=PyPDFLoader)
    documents=loader.load()

    return documents
    

In [5]:
extract_data=load_pdf_file(data='Data/')

In [6]:
def text_split(extract_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extract_data)
    return text_chunks


In [7]:
text_chunks=text_split(extract_data)
print("Length of the text chunks:-", len(text_chunks))

Length of the text chunks:- 7024


In [8]:
from langchain_huggingface import HuggingFaceEmbeddings


In [9]:
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings



In [10]:
embeddings=download_hugging_face_embeddings()
print(embeddings)

c:\Users\KIIT0001\anaconda3\envs\medibot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


model_name='sentence-transformers/all-MiniLM-L6-v2' cache_folder=None model_kwargs={} encode_kwargs={} query_encode_kwargs={} multi_process=False show_progress=False


In [11]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [12]:
from dotenv import load_dotenv
load_dotenv()

True

In [14]:

from langchain.vectorstores import FAISS
docsearch = FAISS.from_documents(
    documents=text_chunks,
    embedding=embeddings
)

# Optional: Save FAISS index locally for reuse
docsearch.save_local("faiss_index")

In [16]:
from langchain.vectorstores import FAISS

docsearch = FAISS.load_local(
    folder_path="faiss_index",
    embeddings=embeddings,
    allow_dangerous_deserialization=True  # ✅ Explicit opt-in
)

In [17]:
docsearch

In [18]:
retriever = docsearch.as_retriever(search_type="similarity", search_kewargs={"k":2})

In [19]:
retrieved_docs=retriever.invoke("What is swelling?")
retrieved_docs

[Document(id='f96fbaaf-1643-498a-a45d-7ad27ca7932b', metadata={'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'creator': 'PyPDF', 'creationdate': '2004-12-18T17:16:32-05:00', 'moddate': '2004-12-18T16:35:04-06:00', 'source': 'Data\\medical_book.pdf', 'total_pages': 759, 'page': 72, 'page_label': '73'}, page_content='swelling. This red, painful region grows bigger as the\ninfection and resulting tissue damage spread. An untreat-\ned infection may spread to the lymphatic system ( acute\nlymphangitis ), the lymph nodes ( lymphadenitis ), the\nbloodstream (bacteremia), or into deeper tissues. Celluli-\ntis most often occurs on the face, neck, and legs.\nOrbital cellulitis\nA very serious infection, called orbital cellulitis,\noccurs when bacteria enter and infect the tissues sur-'),
 Document(id='843fea87-e9df-4521-945a-a9137c3b0d32', metadata={'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'creator': 'PyPDF', 'creationdate': '2004-12-18T17:16:32-05:00', 'moddate': '2004-12-18T16:35:04-06:00', 'source':

In [20]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
import os

In [21]:
GROQ_API_KEY=os.environ.get('GROQ_API_KEY')

In [22]:
llm = ChatGroq(
    model="llama3-70b-8192",
    temperature=0
)

In [23]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)




prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}")
])


question_answer_chain = create_stuff_documents_chain(llm=llm, prompt=prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [24]:
response = rag_chain.invoke({"input": "what is Cirrhosis?"})
print(response["answer"])

Cirrhosis is a liver disease characterized by scarring of the liver tissue, which can lead to liver failure. It can be caused by factors, including poor nutrition, alcoholism, and certain genetic disorders, such as hemochromatosis and Wilson's disease.
